In [1]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [3]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03_fe_v07.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03_fe_v07.parquet...
- [1 files][  5.7 GiB/  5.7 GiB]                                                
Operation completed over 1 objects/5.7 GiB.                                      


In [4]:
base_path = '/home/eanegrin/'
dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03_fe_v07.parquet'

semillas = [122219, 109279, 400391, 401537, 999961]

In [5]:
data = pd.read_parquet(dataset_path + dataset_file)

In [6]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [7]:
meses_excluidos = [202006, 202108, 202109] # meses con clase ternaria incompleta

data = data[~data['foto_mes'].isin(meses_excluidos)]
data['foto_mes'].unique()

array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
       202005, 202007, 202008, 202009, 202010, 202011, 202012, 202101,
       202102, 202103, 202104, 202105, 202106, 202107])

In [8]:
data.shape

(4416394, 1033)

In [9]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201901,0.988730,0.011270
1,201902,0.988808,0.011192
2,201903,0.988548,0.011452
3,201904,0.989464,0.010536
4,201905,0.990287,0.009713
5,201906,0.990169,0.009831
6,201907,0.990071,0.009929
7,201908,0.990691,0.009309
8,201909,0.991594,0.008406
9,201910,0.991169,0.008831


In [10]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    # print(f'Original proportion:{minority_proportion}')
    
    estrategia={0: int(len(y[y == 0]) * 0.02), 
                1: len(y[y == 1])}

    print(f"Se retienen {estrategia[0]} de la clase mayoritaria y {estrategia[1]} de la minoritaria")

    rus = RandomUnderSampler(sampling_strategy=estrategia, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rearmar
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes

    df_subsampled.append(group_resampled)

# Mergear
data = pd.concat(df_subsampled, ignore_index=True)

data.shape

Se retienen 2466 de la clase mayoritaria y 1406 de la minoritaria
Se retienen 2487 de la clase mayoritaria y 1408 de la minoritaria
Se retienen 2499 de la clase mayoritaria y 1448 de la minoritaria
Se retienen 2513 de la clase mayoritaria y 1338 de la minoritaria
Se retienen 2528 de la clase mayoritaria y 1240 de la minoritaria
Se retienen 2558 de la clase mayoritaria y 1270 de la minoritaria
Se retienen 2588 de la clase mayoritaria y 1298 de la minoritaria
Se retienen 2628 de la clase mayoritaria y 1235 de la minoritaria
Se retienen 2663 de la clase mayoritaria y 1129 de la minoritaria
Se retienen 2709 de la clase mayoritaria y 1207 de la minoritaria
Se retienen 2746 de la clase mayoritaria y 1358 de la minoritaria
Se retienen 2786 de la clase mayoritaria y 1332 de la minoritaria
Se retienen 2857 de la clase mayoritaria y 1107 de la minoritaria
Se retienen 2928 de la clase mayoritaria y 693 de la minoritaria
Se retienen 2975 de la clase mayoritaria y 564 de la minoritaria
Se retienen 

(128146, 1033)

In [11]:
# proporciones luego de hacer el ajuste

proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201901,0.636880,0.363120
1,201902,0.638511,0.361489
2,201903,0.633139,0.366861
3,201904,0.652558,0.347442
4,201905,0.670913,0.329087
5,201906,0.668234,0.331766
6,201907,0.665980,0.334020
7,201908,0.680300,0.319700
8,201909,0.702268,0.297732
9,201910,0.691777,0.308223


In [12]:
output_file = 'competencia_03_fe_v07_undersampled.parquet'

data.to_parquet('/home/eanegrin/datasets/' + output_file, index=False)

In [13]:
!gsutil cp /home/eanegrin/datasets/competencia_03_fe_v07_undersampled.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_03_fe_v07_undersampled.parquet...
- [1 files][198.2 MiB/198.2 MiB]                                                
Operation completed over 1 objects/198.2 MiB.                                    
